In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Define file names
rois = [
    "l_p_ins_bn.nii.gz_r_p_ins_bn.nii.gz",
    "l_po_145_bn.nii.gz_r_po_146_bn.nii.gz",
    "l_co_73_bn.nii.gz_r_co_74_bn.nii.gz"
]


custom_titles = [
    "Left Posterior Insula",
    "Right Posterior Insula",
    "Left Parietal Operculum",
    "Right Parietal Operculum",
    "Left Central Operculum",
    "Right Central Operculum"
]


assert len(custom_titles) == 2 * len(rois), "There should be a custom title for each subplot."


fig, axs = plt.subplots(nrows=len(rois), ncols=2, figsize=(50, 20 * len(rois)))

# COPE labels for plotting
cope_labels = ['Warmth', 'Competence', 'Simulation', 'Self-Similarity', 'Universalism']


plt.style.use('ggplot')  

significance_level = 0.05  
plt.rcParams.update({'font.size': 50})  
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'

all_coefficients = []

# Loop over ROIs
for idx, roi in enumerate(rois):
    dims_data = pd.read_csv(f'/Users/akila/Desktop/OB_Data/Results/Rehumanization/Multiple_Regression/fslmeants_results_{roi}_dim_z3.csv')
    bar_width = 0.5  

    pre_post_data = pd.read_csv(f'/Users/akila/Desktop/OB_Data/Results/Rehumanization/Multiple_Regression/fslmeants_results_{roi}_z3.csv')

    # Reshape pre-post Data
    pre_post_data = pre_post_data.pivot(index='Participant', columns='Condition', values=['Mean_ROI1', 'Mean_ROI2'])
    pre_post_data.columns = [f'{i}_{j}' for i, j in pre_post_data.columns]
    pre_post_data['Pre_Post_Change_ROI1'] = pre_post_data['Mean_ROI1_rehum_post'] - pre_post_data['Mean_ROI1_rehum_pre']
    pre_post_data['Pre_Post_Change_ROI2'] = pre_post_data['Mean_ROI2_rehum_post'] - pre_post_data['Mean_ROI2_rehum_pre']

    pre_post_data.reset_index(inplace=True)
    
    title_ROI1 = custom_titles[idx * 2]  # Custom title for ROI1
    title_ROI2 = custom_titles[idx * 2 + 1]  # Custom title for ROI2

    merged_data = pd.merge(dims_data, pre_post_data, on='Participant')

    # COPE dimensions 7 to 11 (each strategy > control)
    filtered_data = merged_data[merged_data['COPE'].isin([7, 8, 9, 10, 11])]

    cope_data = filtered_data.pivot(index='Participant', columns='COPE', values=['Mean_ROI1', 'Mean_ROI2'])
    cope_data.columns = ['{}_COPE{}'.format(*col) for col in cope_data.columns]
    cope_data.reset_index(inplace=True)

    full_data = pd.merge(cope_data, pre_post_data[['Participant', 'Pre_Post_Change_ROI1', 'Pre_Post_Change_ROI2']], on='Participant')

    # Regression analysis 
    dependent_var_ROI1 = full_data['Pre_Post_Change_ROI1']
    dependent_var_ROI2 = full_data['Pre_Post_Change_ROI2']
    independent_vars_ROI1 = full_data[['Mean_ROI1_COPE{}'.format(cope) for cope in range(7, 12)]]
    independent_vars_ROI2 = full_data[['Mean_ROI2_COPE{}'.format(cope) for cope in range(7, 12)]]

    independent_vars_ROI1 = sm.add_constant(independent_vars_ROI1)
    independent_vars_ROI2 = sm.add_constant(independent_vars_ROI2)

    model_ROI1 = sm.OLS(dependent_var_ROI1, independent_vars_ROI1).fit()
    model_ROI2 = sm.OLS(dependent_var_ROI2, independent_vars_ROI2).fit()

    coefficients_ROI1 = model_ROI1.params[1:]
    errors_ROI1 = model_ROI1.bse[1:]  # Standard errors
    coefficients_ROI2 = model_ROI2.params[1:]
    errors_ROI2 = model_ROI2.bse[1:]  # Standard errors

    # Results
    print(f"Group-Level Regression Results for ROI1 - {roi}:\n")
    print(model_ROI1.summary())
    print("\n")
    print(f"Group-Level Regression Results for ROI2 - {roi}:\n")
    print(model_ROI2.summary())
    print("\n\n")

    y_limit = (-0.004, 0.005)

    # Plotting ROI1
    bars_ROI1 = axs[idx, 0].bar(cope_labels, coefficients_ROI1, yerr=errors_ROI1, color=['salmon' if p < significance_level else 'black' for p in model_ROI1.pvalues[1:]], width=bar_width, capsize=10, error_kw=dict(linewidth=3))
    axs[idx, 0].set_title(custom_titles[idx * 2], fontsize=70)
    axs[idx, 0].set_facecolor('white')
    axs[idx, 0].spines['bottom'].set_color('black')
    axs[idx, 0].spines['top'].set_color('black')
    axs[idx, 0].spines['right'].set_color('black')
    axs[idx, 0].spines['left'].set_color('black')
    axs[idx, 0].set_ylim(y_limit)
    axs[idx, 0].set_ylabel('Coefficient Value', labelpad=30, fontsize=70)
    axs[idx, 0].tick_params(axis='x', labelrotation=45, labelsize=60)  # Rotates only x-axis labels
    axs[idx, 0].tick_params(axis='y', labelsize=50)  # Keeps y-axis labels unchanged
    axs[idx, 0].axhline(0, color='blue', linewidth=2)

    for j, (coef, p_value) in enumerate(zip(coefficients_ROI1, model_ROI1.pvalues[1:])):
        y_pos = coef + (0.01 - (-0.01)) * 0.05  # Adjust y position for the asterisk
        if p_value < 0.001:
            axs[idx, 0].text(j, y_pos, "***", ha='center', fontsize=80)
        elif p_value < 0.01:
            axs[idx, 0].text(j, y_pos, "**", ha='center', fontsize=80)
        elif p_value < 0.05:
            axs[idx, 0].text(j, y_pos, "*", ha='center', fontsize=80)

    # Plotting ROI2
    bars_ROI2 = axs[idx, 1].bar(cope_labels, coefficients_ROI2, yerr=errors_ROI2, color=['salmon' if p < significance_level else 'black' for p in model_ROI2.pvalues[1:]], width=bar_width, capsize=10, error_kw=dict(linewidth=3))
    axs[idx, 1].set_title(custom_titles[idx * 2 + 1], fontsize=70)
    axs[idx, 1].set_facecolor('white')
    axs[idx, 1].spines['bottom'].set_color('black')
    axs[idx, 1].spines['top'].set_color('black')
    axs[idx, 1].spines['right'].set_color('black')
    axs[idx, 1].spines['left'].set_color('black')
    axs[idx, 1].set_ylim(y_limit)
    axs[idx, 1].set_ylabel('Coefficient Value', labelpad=30, fontsize=70)
    axs[idx, 1].tick_params(axis='x', labelrotation=45, labelsize=60) 
    axs[idx, 1].tick_params(axis='y', labelsize=50)  
    axs[idx, 1].axhline(0, color='blue', linewidth=2)

    # significance asteriks
    for j, (coef, p_value) in enumerate(zip(coefficients_ROI2, model_ROI2.pvalues[1:])):
        y_pos = coef + (0.01 - (-0.01)) * 0.05  
        if p_value < 0.001:
            axs[idx, 1].text(j, y_pos, "***", ha='center', fontsize=80)
        elif p_value < 0.01:
            axs[idx, 1].text(j, y_pos, "**", ha='center', fontsize=80)
        elif p_value < 0.05:
            axs[idx, 1].text(j, y_pos, "*", ha='center', fontsize=80)

   
    y_ticks = np.linspace(y_limit[0], y_limit[1], 6) 
    axs[idx, 0].set_yticks(y_ticks)
    axs[idx, 1].set_yticks(y_ticks)

    # visual appearance
    for spine in axs[idx, 0].spines.values():
        spine.set_linewidth(2)  
    for spine in axs[idx, 1].spines.values():
        spine.set_linewidth(2)  

plt.tight_layout(pad=7.0)  

# save fig
fig.savefig("plot_output.png", format='png')
fig.savefig("plot_output.svg", format='svg')

plt.show()
